In [ ]:
!rm -rf AtomML-Course
!git clone https://github.com/AMLS-PRG/AtomML-Course

Cloning into 'AtomML-Course'...
remote: Enumerating objects: 2751, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 2751 (delta 87), reused 21 (delta 21), pack-reused 2625 (from 1)
Receiving objects: 100% (2751/2751), 47.27 MiB | 20.98 MiB/s, done.
Resolving deltas: 100% (1356/1356), done.


In [ ]:
pip install ase numpy matplotlib
import numpy as np
import matplotlib.pyplot as plt
from ase.io.lammpsrun import read_lammps_dump
from ase.geometry.analysis import Analysis

In [ ]:
# ----------- Parameters -----------
file_liquid = "AtomML-Course/module-5/04-Performing-MD-simulations/runLammps/md.dump"
file_solid  = "AtomML-Course/module-5/04-Performing-MD-simulations/runLammps/md_solid.dump"

n_frames = 500              # use last 500 frames
time_step_fs = 100          # 100 fs per frame
bins = np.arange(5, 51, 5)  # bin sizes: 5, 10, ..., 50 frames (0.5 ps to 5 ps)

In [ ]:
# ----------- Helper: unwrap PBC for ASE trajectory -----------
def unwrap_positions(frames):
    """
    Reconstruct atom trajectories with unwrapped positions using PBC and LAMMPS image flags.
    """
    unwrapped = []
    positions0 = frames[0].positions.copy()
    box = frames[0].get_cell()

    # LAMMPS image flags (ix, iy, iz) store how many times an atom has crossed a boundary
    for atoms in frames:
        if 'ix' not in atoms.arrays:
            raise ValueError("LAMMPS dump must include image flags (ix, iy, iz) for unwrapping.")
        shift = np.dot(atoms.arrays[['ix', 'iy', 'iz']], box)
        pos_unwrapped = atoms.positions + shift
        unwrapped.append(pos_unwrapped)
    return np.array(unwrapped)  # shape: (n_frames, n_atoms, 3)

# ----------- Compute MSD -----------

def compute_msd(frames, bins, time_step_fs):
    """
    Compute MSD from a trajectory of ASE Atoms objects for atom type 1.

    Returns:
    - times_fs: time lag in fs
    - msd_vals: array of MSD values for each bin
    """
    # Get unwrapped positions of atoms of type 1
    type_mask = frames[0].arrays["type"] == 1
    unwrapped_all = unwrap_positions(frames)[:, type_mask, :]

    n_atoms = unwrapped_all.shape[1]
    n_total = unwrapped_all.shape[0]

    msd_vals = []
    times_fs = []

    for dt in bins:
        displacements = unwrapped_all[dt:] - unwrapped_all[:-dt]
        sq_disp = np.square(displacements).sum(axis=2)  # (frames - dt, atoms)
        msd_dt = np.mean(sq_disp)
        msd_vals.append(msd_dt)
        times_fs.append(dt * time_step_fs)

    return np.array(times_fs), np.array(msd_vals)

In [ ]:
# ----------- Load last 500 frames -----------
frames_liquid = read_lammps_dump(file_liquid, index=slice(-n_frames, None))
frames_solid  = read_lammps_dump(file_solid, index=slice(-n_frames, None))

# ----------- Compute MSD -----------
times_liq, msd_liq = compute_msd(frames_liquid, bins, time_step_fs)
times_sol, msd_sol = compute_msd(frames_solid,  bins, time_step_fs)

In [ ]:
# ----------- Plot MSD -----------

plt.figure(figsize=(8, 5))
plt.plot(times_liq / 1000, msd_liq, 'o--', label='Liquid')  # time in ps
plt.plot(times_sol / 1000, msd_sol, 's--', label='Solid')
plt.xlabel("Time lag (ps)", fontsize=12)
plt.ylabel("MSD (Å²)", fontsize=12)
plt.title("Mean Squared Displacement (MSD)", fontsize=14)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("msd_comparison.png")
plt.show()